In [1]:
#importing the libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [2]:
# Upload the file to Collab
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up"],\r\n         "responses": ["Hello!", "Good to see you again!", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["cya", "See you later", "Goodbye", "I am Leaving", "Have a Good day"],\r\n         "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "age",\r\n         "patterns": ["how old", "how old is tim", "what is your age", "how old are you", "age?"],\r\n         "responses": ["I am 18 years old!", "18 years young!"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "name",\r\n         "patterns": ["what is your name", "what should I call you", "whats your name?"],\r\n         "responses": ["You can call me Tim.", "I\'m Tim!", "I\'m Tim aka Tech With

In [4]:
#importing the dataset
with open('intents.json') as content:
  data1 = json.load(content)
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['patterns']:
    inputs.append(lines)
    tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})
print(data)

                          inputs      tags
0                             Hi  greeting
1                    How are you  greeting
2               Is anyone there?  greeting
3                          Hello  greeting
4                       Good day  greeting
5                       Whats up  greeting
6                            cya   goodbye
7                  See you later   goodbye
8                        Goodbye   goodbye
9                   I am Leaving   goodbye
10               Have a Good day   goodbye
11                       how old       age
12                how old is tim       age
13              what is your age       age
14               how old are you       age
15                          age?       age
16             what is your name      name
17        what should I call you      name
18              whats your name?      name
19      Id like to buy something      shop
20             whats on the menu      shop
21       what do you reccommend?      shop
22  could i

In [5]:
#removing punctuations
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [6]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)


6
number of unique words :  48
output length:  6


In [7]:
#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
1/1 [==============================] - 0s 2ms/step - loss: 1.7906 - accuracy: 0.1923
Epoch 2/200
1/1 [==============================] - 0s 1ms/step - loss: 1.7890 - accuracy: 0.1923
Epoch 3/200
1/1 [==============================] - 0s 1ms/step - loss: 1.7875 - accuracy: 0.2692
Epoch 4/200
1/1 [==============================] - 0s 1ms/step - loss: 1.7859 - accuracy: 0.2308
Epoch 5/200
1/1 [==============================] - 0s 2ms/step - loss: 1.7844 - accuracy: 0.2308
Epoch 6/200
1/1 [==============================] - 0s 2ms/step - loss: 1.7828 - accuracy: 0.2308
Epoch 7/200
1/1 [==============================] - 0s 2ms/step - loss: 1.7813 - accuracy: 0.2308
Epoch 8/200
1/1 [==============================] - 0s 2ms/step - loss: 1.7797 - accuracy: 0.2308
Epoch 9/200
1/1 [==============================] - 0s 2ms/step - loss: 1.7781 - accuracy: 0.2308
Epoch 10/200
1/1 [==============================] - 0s 3ms/step - loss: 1.7765 - accuracy: 0.2308
Epoch 11/200
1/1 [===========

In [11]:
#chatting
import random
while True:
  texts_p = []
  prediction_input = input('You : ')
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)
  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()
  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print("Going ChatBot : ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

You : Hi
Going Merry :  Hi there, how can I help?
You : How old are you
Going Merry :  18 years young!
You : Goodbye
Going Merry :  Sad to see you go :(


In [13]:
import pickle
import joblib

In [14]:
model.save('intents_chatbot_tf2_lstm.h5')

In [15]:
joblib.dump(tokenizer,'token_chatbot.pkl')

['token_chatbot.pkl']